# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Wed Mar  3 23:21:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro M2200        Off  | 00000000:01:00.0  On |                  N/A |
| N/A   46C    P0    N/A /  N/A |    687MiB /  4043MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.config.list_physical_devices('XLA_GPU'))
device_lib.list_local_devices()

[]


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4654194786491279941,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2817720320
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9692943512775352870
 physical_device_desc: "device: 0, name: Quadro M2200, pci bus id: 0000:01:00.0, compute capability: 5.2"]

In [4]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.2321, 0.9498, 0.4008],
        [0.3944, 0.2666, 0.6754],
        [0.9186, 0.5918, 0.9133],
        [0.7820, 0.7311, 0.1419],
        [0.4469, 0.3941, 0.9339]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [5]:
x = np.random.rand(10000, 256)

In [6]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

28 s ± 74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [7]:
x = torch.rand(10000, 256)

In [8]:
%%timeit
# Calculate the projection matrix of x on the CPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

202 ms ± 4.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 3) PyTorch on GPU via CUDA

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.7952, 0.3012, 0.2262, 0.9393, 0.3986],
        [0.8615, 0.7097, 0.2090, 0.0741, 0.2877],
        [0.7233, 0.6337, 0.3760, 0.4210, 0.4026],
        [0.2837, 0.3296, 0.2638, 0.8314, 0.0068],
        [0.1824, 0.9997, 0.3499, 0.1608, 0.8785]], device='cuda:0')
tensor([[0.7952, 0.3012, 0.2262, 0.9393, 0.3986],
        [0.8615, 0.7097, 0.2090, 0.0741, 0.2877],
        [0.7233, 0.6337, 0.3760, 0.4210, 0.4026],
        [0.2837, 0.3296, 0.2638, 0.8314, 0.0068],
        [0.1824, 0.9997, 0.3499, 0.1608, 0.8785]], dtype=torch.float64)


In [10]:
%%timeit
# Calculate the projection matrix of x on the GPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

40.1 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Exhaustive Testing on GPU

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [12]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.7547, 0.4407, 0.3324, 0.7541, 0.4007],
        [0.3266, 0.7275, 0.1081, 0.7451, 0.9306],
        [0.0367, 0.0941, 0.0734, 0.2988, 0.6229],
        [0.5275, 0.9039, 0.0472, 0.6376, 0.6114],
        [0.6264, 0.8484, 0.3481, 0.6612, 0.7368]], device='cuda:0')


In [13]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border. 
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [14]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 7.2284e-04,  6.6641e-05, -6.3184e-05,  1.6315e-04,  4.2660e-04],
        [ 6.6641e-05,  9.5519e-04,  1.2575e-04,  2.9232e-04,  9.9097e-05],
        [-6.3184e-05,  1.2575e-04,  5.4746e-04,  3.1664e-04,  3.1305e-04],
        [ 1.6315e-04,  2.9231e-04,  3.1664e-04,  1.0772e-03,  4.2015e-04],
        [ 4.2660e-04,  9.9097e-05,  3.1305e-04,  4.2015e-04,  8.0372e-04]],
       device='cuda:0')


In [15]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred 
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 7.2284e-04,  6.6641e-05, -6.3184e-05,  1.6315e-04,  4.2660e-04],
        [ 6.6641e-05,  9.5519e-04,  1.2575e-04,  2.9232e-04,  9.9097e-05],
        [-6.3184e-05,  1.2575e-04,  5.4746e-04,  3.1664e-04,  3.1305e-04],
        [ 1.6315e-04,  2.9231e-04,  3.1664e-04,  1.0772e-03,  4.2015e-04],
        [ 4.2660e-04,  9.9097e-05,  3.1305e-04,  4.2015e-04,  8.0372e-04]],
       dtype=torch.float64)
